In [1]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import sisepuede_file_structure as sfs
import support_classes as sc
import support_functions as sf
import importlib
import sql_utilities as squ
import time
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2029: UserWarning: Warning passed from optional_log: Properties for variable :math:\text{CH}_4 Stationary Combustion Emission Factor successfully set with 0 errors...
  warnings.warn(f"Warning passed from optional_log: {msg}.")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2029: UserWarning: Warning passed from optional_log: Properties for variable :math:\text{CO}_2 Combustion Emission Factor successfully set with 0 errors...
  warnings.warn(f"Warning passed from optional_log: {msg}.")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2029: UserWarning: Warning passed from optional_log: Properties for variable :math:\text{N}_2\text{O} Stationary Combustion Emission Factor successfully set with 0 errors...
  warnings.warn(f"Warning passed from optional_log: {msg}.")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:2029

MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


In [2]:
from analysis_id import *
aid = AnalysisID(
    regex_template = re.compile("sisepuede_inputs_(.+$)")
)

In [3]:
import iterative_database as idb
import sqlalchemy

fp = f"/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data_pipeline/{aid.id_fs_safe}.sqlite"
engine = sqlalchemy.create_engine(f"sqlite:///{fp}")

db = idb.IterativeDatabase(
    engine,
    None,
    #analysis_id = aid,
)

dict_def = {
    "cropland_fraction": {
        "fields_index": [
            "region", 
            "year",
        ]
    }
}

db._add_tables(
    dict_def,
    overwrite_existing_if_in_dict_idt = False,
)

In [4]:
db.engine

Engine(sqlite:////Users/jsyme/Documents/Projects/git_jbus/sisepuede_data_pipeline/sisepuede_inputs_2024-03-13T01;42;09.595435.sqlite)

In [5]:
fp_config = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/sisepuede_data_construction/soil/fertilizer/config.yaml"

config = sc.YAMLConfiguration(fp_config)
config.get("inputs.ifa_plant_nutrition.local")


In [6]:
fp_lib = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data_pipeline"
import sys
if fp_lib not in sys.path:
    sys.path.append(fp_lib)
import lib.data_objects as do
import lib.process_utilities as pu
import lib.sisepuede_data_constructs as dc
import lib.data_construction_utilities as util

In [7]:
import afolu.initial_crops_and_land_use.fao_cropland_proportions as props
importlib.reload(ma)
importlib.reload(sfs)
importlib.reload(dc)
importlib.reload(do)
importlib.reload(sf)
importlib.reload(util)
importlib.reload(pu)
importlib.reload(props)

<module 'afolu.initial_crops_and_land_use.fao_cropland_proportions' from '/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data_pipeline/afolu/initial_crops_and_land_use/fao_cropland_proportions.py'>

In [41]:
fp_config = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data_pipeline/afolu/initial_crops_and_land_use/config.yaml"
reader = pu.InputsReader(fp_config)
construct = dc.SISEPUEDEDataConstructs()



MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


In [17]:
#reader.config.get("inputs.fao_land_cover.element")
modvar_target = "Initial Land Use Area Proportion"
modvar_target_var = sa.model_attributes.get_variable(modvar_target)
dict_cats_to_fields = sa.model_attributes.get_category_replacement_field_dict(modvar_target)


In [60]:
importlib.reload(sf)
importlib.reload(sc)
regions = sc.Regions(sa.model_attributes)


df_check_out = regions.convert_region_codes(
    reader.fao_land_cover.df,
    "Area Code",
    "fao_region_code",
    "iso"
)


NameError: name 'field_codes' is not defined

In [19]:
df_check = reader.fao_land_cover.df.copy()

In [57]:
def get_elements_from_element_spec(
    vec: Union[List[str], np.ndarray, pd.Series],
    elements: str,
    delim: str = "|",
    flag_all: str = "all",
) -> List[str]:
    """
    Check the specification of elements in the set vec. Delimiter `delim` is used to 
        split elements specified in the string
    """
    
    all_elements = set(vec)
    all_elements = sorted(list(all_elements))
    
    if elements == flag_all:
        return all_elements
    
    
    elements_try = elements.split(delim)
    elements_try = [x for x in elements_try if x in all_elements]
    
    return elements_try
        
def get_land_cover_data2(
    fao_obj: do.DatasetFAO,
    element_spec: str,
    dict_cats_to_fields: dict,
    regions: sc.Regions, 
    time_periods: sc.TimePeriods,
    model_attributes: ma.ModelAttributes,
    delim_elements: str = "|",
    field_fraction: str = "frac",
    flag_all: str = "all",
) -> pd.DataFrame: 
    """
        Process Land Cover data, the primary source of initial SISEPUEDE land use
            prevalence proportions

        Function Arguments
        ------------------
        - fao_obj: DatasetFAO object that stores Land Cover data and the crosswalk
        - element_spec: FAO Element--used to specify which land cover dataset to use for estimates
            NOTE: specify `flag_all` to take the mean across all available datasets OR a delim_elements delimited list 
        - dict_cats_to_fields: dictionary mapping SISEPUEDE Land Use categories to fields
        - regions: regions object used to map FAO codes to region name
        - time_periods: used for time period management (including fields)
        - model_attributes: SISEPUEDE model attributes object used to access 
            variable information.

        Keyword Arguments
        -----------------
        - delim_elements: delimter used to divide specifications of elements to keep
        - flag_all: flag used to determine that all elements should be kept
    """
    return None



def get_land_cover_data(
    fao_obj: do.DatasetFAO,
    element_spec: str,
    regions: sc.Regions, 
    delim_elements: str = "|",
    flag_all: str = "all",
) -> pd.DataFrame:
    """
    Process Land Cover data, the primary source of initial SISEPUEDE land use
        prevalence proportions. 
        
        - Filters on the element (method) used to estimate land cover and aggregates as necessary
        - Converts FAO region code to ISO Alpha 3
        - Converts FAO land use categories to SISEPUEDE categories (or those used
            for estimating)
        
    Function Arguments
    ------------------
    - fao_obj: DatasetFAO object that stores Land Cover (as `fao_land_cover`) data AND the crosswalk (as `fao_crosswalk_land_cover`) containing SISEPUEDE categories
    - element_spec: FAO Element--used to specify which land cover dataset to use for estimates
        NOTE: specify `flag_all` to take the mean across all available datasets OR a delim_elements delimited list 
    - dict_cats_to_fields: dictionary mapping SISEPUEDE Land Use categories to fields
    - regions: regions object used to map FAO codes to region name
    - time_periods: used for time period management (including fields)
    - model_attributes: SISEPUEDE model attributes object used to access 
        variable information.

    Keyword Arguments
    -----------------
    - delim_elements: delimter used to divide specifications of elements to keep
    - flag_all: flag used to determine that all elements should be kept
    """
    
    # get the DatasetFAO object (datalc) + crosswalk
    datalc = fao_obj.fao_land_cover
    df_items_cw = fao_obj.fao_crosswalk_land_cover
    
    # configuration can set which estimation source we want to use
    elements = get_elements_from_element_spec(
        element_spec,
        delim = delim_elements,
    )
    
    # use the try block so that we can (a) catch and (b) track where errors originate from in the system
    try:
        df_lc = (
            datalc.get_subset(
                element = elements,
            )[
                [
                    fao_obj.field_area_code,
                    fao_obj.field_year,
                    fao_obj.field_item,
                    fao_obj.field_unit,
                    fao_obj.field_value
                ]
            ]
        )

    except Exception as e:
        msg = f"Error retrieving FAO land cover data for {modvar_target}: {e}"
        raise RuntimeError(msg)
    
    
    ##  RUN SOME CHECKS
    
    # check to make sure there are valid data
    if len(df_areas) == 0:
        msg = f"Error retrieving {element} for model variable {modvar_target} from FAO land cover data: no rows found in the dataset."
        raise RuntimeError(msg)
    
    # verify that there the units are unique
    units_unique = df_lc[fao_obj.field_unit].unique()
    if len(units_unique):
        msg = f"Error in FAO land cover data: multiple units found in the dataset. Write a routine to convert units using model_attributes.get_unit('area')."
        raise RuntimeError(msg)
    
    
    ##  PREPARE LAND COVER
    
    # drop units, we know there's a unique error if it passed the check
    df_lc.drop(
        [fao_obj.field_unit], 
        axis = 1, 
        inplace = True,
    )
    
    # next, aggregate if desired
    if len(elements) > 1:
        
        fields_group = [x for x in df_lc.columns if x != fao_obj.value]
        dict_agg = dict((x, "first") for x in fields_group)
        dict_agg.update({fao_obj.value: "mean"})
        
        
        df_lc = sf.simple_df_agg(
            df_lc,
            fields_group,
            dict_agg
        )
        
    # drop the element field
    df_lc.drop(
        [fao_obj.field_element], 
        axis = 1, 
        inplace = True,
    )
        
    # finally, replace the fao code with the region iso
    df_lc = regions.convert_region_codes(
        df_lc,
        fao_obj.field_area_code,
        "fao_region_code",
        "iso" # uses regions.field_iso
    )
    
    return df_lc
        

get_land_cover_data(
    reader.fao_land_cover,
    
)      
    

In [51]:
sa.model_attributes.get_subsector_attribute("Land Use", "primary_category")




'``$CAT-LANDUSE$``'

In [57]:
#util.get_fao_crosswalk_dictionary_from_df(reader.fao_crosswalk_crop_type, sa.model_attributes)



In [32]:
"""
fp_read = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/out/sisepuede_summary_results_run_sisepuede_run_2023-09-28T11;36;58.322719/sisepuede_results_sisepuede_run_2023-09-28T11;36;58.322719_WIDE_INPUTS_OUTPUTS.csv"
df_read = pd.read_csv(fp_read)
df1 = df_read[
    df_read["region"].isin(["brazil", "mexico", "dominican_republic"])
    #df_read["region"].isin([0, 14014, 119119]
    & df_read["time_period"].isin([0, 35])
][
    ["primary_id", "region", "emission_co2e_co2_frst_harvested_wood_products", "emission_co2e_subsector_total_frst"]
    #+ [x for x in df_read.columns if x.startswith("emission_co2e_subsector_total")]
]
"""


In [42]:
#regions.attributes.table
reader.fao_crosswalk_land_cover

,Item,``$CAT-LANDUSE$``
0,Artificial surfaces (including urban and assoc...,settlements
1,Herbaceous crops,croplands
2,Woody crops,croplands
3,Multiple or layered crops,croplands
4,Grassland,grasslands
5,Tree-covered areas,forests_agg
6,Mangroves,forests_mangroves
7,Shrub-covered areas,grasslands
8,"Shrubs and/or herbaceous vegetation, aquatic o...",other
9,Sparsely natural vegetated areas,other


In [61]:
dir(regions)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_initialize_defaults_iea',
 '_initialize_generic_dict',
 '_initialize_region_properties',
 'add_region_or_iso_field',
 'aggregate_df_by_wb_global_region',
 'all_isos',
 'all_isos_numeric',
 'all_regions',
 'all_un_regions',
 'all_wb_regions',
 'attributes',
 'clean_region',
 'data_func_iea_get_isos_from_countries',
 'data_func_try_isos_from_countries',
 'dict_generic_countries_to_regions',
 'dict_iea_countries_lc_to_regions',
 'dict_iso_numeric_to_region',
 'dict_iso_to_region',
 'dict_region_to_iso',
 'dict_region_to_iso_numeric',
 'dict_region_to_un_region',
 'dict_region_to_wb_region',
 'dict_un_regio

In [13]:
importlib.reload(pu)
importlib.reload(props)


tup = props.main(
    inputs_reader = reader,
    return_behavior = "write_only",
    sisepuede_constructs = construct,
)

In [ ]:
engine = construct.output_database.engine
squ.sql_table_to_df(
    engine,
    "INITIAL_CROPLAND_AREA_PROPORTION_HISTORICAL"
)

In [49]:
fp_dat = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT2024/Environment_LandCover_E_All_Data_(Normalized)/Environment_LandCover_E_All_Data_(Normalized).csv"

dataset = do.DatasetFAO(fp_dat)


In [47]:
df_lu = pd.read_csv(
    "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/dev/pipeline_examples/Initial Land Use Area Proportion/row_data/Land_cover_data/FAOSTAT_data_7-11-2022.csv"
)

In [67]:
dataset.get_subset(year = 2020, area = "Afghanistan")

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,'004,Afghanistan,6970,Artificial surfaces (including urban and assoc...,5007,Area from MODIS,2020,2020,1000 ha,119.44,E
1,2,'004,Afghanistan,6970,Artificial surfaces (including urban and assoc...,5008,Area from CCI_LC,2020,2020,1000 ha,73.42,E
2,2,'004,Afghanistan,6970,Artificial surfaces (including urban and assoc...,5013,Area from WorldCover,2020,2020,1000 ha,236.97,E
3,2,'004,Afghanistan,6971,Herbaceous crops,5007,Area from MODIS,2020,2020,1000 ha,1493.92,E
4,2,'004,Afghanistan,6971,Herbaceous crops,5008,Area from CCI_LC,2020,2020,1000 ha,5708.61,E
5,2,'004,Afghanistan,6971,Herbaceous crops,5013,Area from WorldCover,2020,2020,1000 ha,4493.40,E
6,2,'004,Afghanistan,6972,Woody crops,5007,Area from MODIS,2020,2020,1000 ha,NaN,O
7,2,'004,Afghanistan,6972,Woody crops,5008,Area from CCI_LC,2020,2020,1000 ha,162.73,E
8,2,'004,Afghanistan,6972,Woody crops,5013,Area from WorldCover,2020,2020,1000 ha,NaN,O
9,2,'004,Afghanistan,6973,Multiple or layered crops,5007,Area from MODIS,2020,2020,1000 ha,NaN,O


In [63]:
df_lu[
    df_lu["Area"].isin(["Afghanistan"])
    & df_lu["Year"].isin([2019])
]

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Unnamed: 14,Unnamed: 15
4,LC,Land Cover,2,Afghanistan,5006,Area from CGLS,6970,Artificial surfaces (including urban and assoc...,2019,2019,1000 ha,108.3364,FC,Calculated data,NaN,NaN
13,LC,Land Cover,2,Afghanistan,5007,Area from MODIS,6970,Artificial surfaces (including urban and assoc...,2019,2019,1000 ha,90.4142,FC,Calculated data,NaN,NaN
22,LC,Land Cover,2,Afghanistan,5008,Area from CCI_LC,6970,Artificial surfaces (including urban and assoc...,2019,2019,1000 ha,73.4567,FC,Calculated data,NaN,NaN
27,LC,Land Cover,2,Afghanistan,5006,Area from CGLS,6971,Herbaceous crops,2019,2019,1000 ha,4392.2500,FC,Calculated data,NaN,NaN
36,LC,Land Cover,2,Afghanistan,5007,Area from MODIS,6971,Herbaceous crops,2019,2019,1000 ha,1458.7517,FC,Calculated data,NaN,NaN
45,LC,Land Cover,2,Afghanistan,5008,Area from CCI_LC,6971,Herbaceous crops,2019,2019,1000 ha,5709.2968,FC,Calculated data,NaN,NaN
50,LC,Land Cover,2,Afghanistan,5006,Area from CGLS,6972,Woody crops,2019,2019,1000 ha,NaN,NV,Data not available,NaN,NaN
59,LC,Land Cover,2,Afghanistan,5007,Area from MODIS,6972,Woody crops,2019,2019,1000 ha,NaN,NV,Data not available,NaN,NaN
68,LC,Land Cover,2,Afghanistan,5008,Area from CCI_LC,6972,Woody crops,2019,2019,1000 ha,162.7457,FC,Calculated data,NaN,NaN
73,LC,Land Cover,2,Afghanistan,5006,Area from CGLS,6973,Multiple or layered crops,2019,2019,1000 ha,NaN,NV,Data not available,NaN,NaN


In [8]:
dictk = {"dataset": "faostat"}
pu.util.csv_to_data(
    '/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT2024/Emissions_Totals_E_All_Data_(Normalized)/Emissions_Totals_E_All_Data_(Normalized).csv',
    **dictk
)

here
{'csv': {'encoding': 'ISO-8859-1'}}


In [65]:
importlib.reload(sf)
importlib.reload(do)
importlib.reload(util)
importlib.reload(ir)
fp_config = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data_pipeline/afolu/fertilizer/config.yaml"

inputs = pu.InputsReader(
    fp_config
)

In [83]:
importlib.reload(paths)
warnings.filterwarnings("ignore")
constructor = paths.SISEPUEDEDataConstructs()

MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


In [2]:
# read in fertilizer data from IFA Database
df_data = pd.read_excel("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/AFOLU/IFADATA Plant Nutrition query - 06-Aug-2022_01.30.xlsx", skiprows = 2)

# check forest products
encode = "ISO-8859-1"
dir_faostat = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/"
df_frst_emission = pd.read_csv(os.path.join(dir_faostat, "Emissions_Totals_E_All_Data_(Normalized)", "Emissions_Totals_E_All_Data_(Normalized).csv"),
    encoding = encode
)
df_ag_production = pd.read_csv(os.path.join(dir_faostat, "Production_Crops_Livestock_E_All_Data_(Normalized)", "Production_Crops_Livestock_E_All_Data_(Normalized).csv"),
    encoding = encode
)


/var/folders/8m/3ll2cn6d1hdcs6gjqxr2jx5d2hffc9/T/ipykernel_41196/648536644.py:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_frst_emission = pd.read_csv(os.path.join(dir_faostat, "Emissions_Totals_E_All_Data_(Normalized)", "Emissions_Totals_E_All_Data_(Normalized).csv"),


In [18]:
import importlib
importlib.reload(sc)
regions = sc.Regions(sa.model_attributes)

regions.return_region_or_iso(
    list(df_data.dropna()["Country"].unique()),
    clean_inputs = True,
)

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1351: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1351: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:


['morocco',
 'libya',
 'jordan',
 'pakistan',
 'norway',
 'ireland',
 'argentina',
 'spain',
 None,
 'trinidad_and_tobago',
 'romania',
 'france',
 'nigeria',
 'croatia',
 'serbia',
 'kuwait',
 'kazakhstan',
 'viet_nam',
 'kyrgyzstan',
 None,
 'lebanon',
 'tunisia',
 'qatar',
 'north_macedonia',
 'myanmar',
 'venezuela',
 'sri_lanka',
 'germany',
 'indonesia',
 'switzerland',
 'italy',
 'saudi_arabia',
 'cameroon',
 'nepal',
 'finland',
 'bangladesh',
 'zimbabwe',
 'greece',
 'ecuador',
 'iceland',
 None,
 'costa_rica',
 'china',
 None,
 'iran',
 'india',
 'sudan',
 'chile',
 'colombia',
 None,
 None,
 'mexico',
 'nicaragua',
 'estonia',
 'senegal',
 'austria',
 'new_zealand',
 'south_africa',
 'azerbaijan',
 'belarus',
 'latvia',
 'iraq',
 'bulgaria',
 'brazil',
 'lithuania',
 'georgia',
 'kenya',
 'poland',
 'czechia',
 None,
 'slovenia',
 'zambia',
 'israel',
 'peru',
 'denmark',
 'cuba',
 'mauritius',
 'turkmenistan',
 'bosnia_and_herzegovina',
 None,
 'ukraine',
 'portugal',
 'mal

In [13]:
regions = sc.Regions(sa.model_attributes)
[regions.return_region_or_iso(x) for x in l]
list(df_data.dropna()["Country"].unique())

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1351: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1351: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:


['Morocco',
 'Libya',
 'Jordan',
 'Pakistan',
 'Norway',
 'Ireland',
 'Argentina',
 'Spain',
 'Korea DPR',
 'Trinidad and Tobago',
 'Romania',
 'France',
 'Nigeria',
 'Croatia',
 'Serbia',
 'Kuwait',
 'Kazakhstan',
 'Viet Nam',
 'Kyrgyzstan',
 'Tanzania',
 'Lebanon',
 'Tunisia',
 'Qatar',
 'North Macedonia',
 'Myanmar',
 'Venezuela',
 'Sri Lanka',
 'Germany',
 'Indonesia',
 'Switzerland',
 'Italy',
 'Saudi Arabia',
 'Cameroon',
 'Nepal',
 'Finland',
 'Bangladesh',
 'Zimbabwe',
 'Greece',
 'Ecuador',
 'Iceland',
 'Belgium and Luxemburg',
 'Costa Rica',
 'China',
 'Taiwan, China',
 'Iran',
 'India',
 'Sudan',
 'Chile',
 'Colombia',
 "Côte d'Ivoire",
 'U.S.A.',
 'Mexico',
 'Nicaragua',
 'Estonia',
 'Senegal',
 'Austria',
 'New Zealand',
 'South Africa',
 'Azerbaijan',
 'Belarus',
 'Latvia',
 'Iraq',
 'Bulgaria',
 'Brazil',
 'Lithuania',
 'Georgia',
 'Kenya',
 'Poland',
 'Czechia',
 'Russian Federation',
 'Slovenia',
 'Zambia',
 'Israel',
 'Peru',
 'Denmark',
 'Cuba',
 'Mauritius',
 'Turkm

In [ ]:
d1 = df_frst_emission[
    df_frst_emission["Item"].isin(["Agricultural Soils"]) & 
    df_frst_emission["Area"].isin(["Argentina"])# & 
    #df_frst_emission["Element"].isin(["Emissions (CO2eq) (AR5)"])
]
d1["Value"].plot()
d1["Element"].unique()

In [165]:


##  format the regional data
def format_regional_data(
    df_data_by_region: pd.DataFrame,
    fields_ind: list = ["Country", "Year"],
    fields_values: list = ["Consumption"],
    fields_pivot: list = ["Product"],
    dict_rnm_out: dict = None
) -> pd.DataFrame:
    
    # pivot the data
    df_data_by_region_wide = pd.pivot(df_data_by_region, index = fields_ind, columns = fields_pivot, values = ["Consumption"]).reset_index()
    df_data_by_region_wide.columns = df_data_by_region_wide.columns.to_flat_index()

    # rename
    dict_rnm = dict([((a, x), x.lower().replace("(", "").replace(")", "").replace(" ", "_")) for x in keep_products for a in fields_values])
    dict_rnm.update(
        dict([((x, ""), x.lower()) for x in fields_ind])
    )
    df_data_by_region_wide.rename(columns = dict_rnm, inplace = True)

    # clean some columns
    df_data_by_region_wide["year"] = df_data_by_region_wide["year"].astype(int)
    
    if dict_rnm is not None:
        df_data_by_region_wide.rename(columns = dict_rnm_out, inplace = True)
        
    return df_data_by_region_wide




# some sets
keep_products = set(["Grand Total N", "Urea (N)"])
attr_region = sa.model_attributes.get_other_attribute_table(sa.model_attributes.dim_region)
regions_all = set(attr_region.table["category_name"])
regions_available = set(df_data["Country"])
regions_to_allocate = regions_all - regions_available
region_to_allocate = "Others Latin America"

# reduce dataset - piece out to data available bty country and values to allocate proportionally to ag production
df_data = df_data[df_data["Product"].isin(keep_products)].reset_index(drop = True)
df_data_by_region = df_data[df_data["Country"].isin(regions_all)].copy().reset_index(drop = True)
df_data_to_allocate = df_data[df_data["Country"].isin([region_to_allocate])].copy().reset_index(drop = True)

# format fields
field_qty = "qtyinit_soil_synthetic_fertilizer_kt"
field_frac = "frac_soil_synthetic_fertilizer_urea"
dict_rnm = {
    "grand_total_n": field_qty,
    "urea_n": field_frac
}
# get data for which information is available per region
df_data_by_region_wide = format_regional_data(df_data_by_region, dict_rnm_out = dict_rnm)
df_data_by_region_wide[field_frac] = np.array(df_data_by_region_wide[field_frac])/np.array(df_data_by_region_wide[field_qty])


##  do the allocation to countries for which fertilizer data is not available

# get production area by country to allocate fertilizer
df_ag_production_agg = df_ag_production[
    df_ag_production["Element"].isin(["Area harvested"]) &
    df_ag_production["Year"].isin(range(2015, 2020))
][["Area", "Item", "Year", "Value"]];
# grouping fields
fields_grp = ["Area", "Year"]
fields_sum = ["Value"]
dict_agg = dict(zip(fields_grp, ["first" for x in fields_grp]))
dict_agg.update(dict(zip(fields_sum, ["sum" for x in fields_sum])))
# aggregation
df_ag_production_agg = df_ag_production_agg.groupby(fields_grp).agg(dict_agg).reset_index(drop = True)
df_ag_production_agg["Area"] = df_ag_production_agg["Area"].replace({"Bolivia (Plurinational State of)": "Bolivia"})
df_ag_production_agg = df_ag_production_agg[df_ag_production_agg["Area"].isin(regions_to_allocate)].reset_index(drop = True)
# get total area and merge back in, then get fractional allocation
df_ag_production_agg_total = df_ag_production_agg[["Year", "Value"]].groupby(["Year"]).agg({"Year": "first", "Value": "sum"}).reset_index(drop = True).rename(columns = {"Value": "Value_Total"})
df_ag_production_agg = pd.merge(df_ag_production_agg, df_ag_production_agg_total)
df_ag_production_agg["frac_allocation"] = np.array(df_ag_production_agg["Value"])/np.array(df_ag_production_agg["Value_Total"])

# setup the allocation
df_data_to_allocate_wide = format_regional_data(df_data_to_allocate, dict_rnm_out = dict_rnm)
df_data_to_allocate_wide[field_frac] = np.array(df_data_to_allocate_wide[field_frac])/np.array(df_data_to_allocate_wide[field_qty])
df_data_to_allocate_wide.drop(["country"], axis = 1, inplace = True)
# merge in production data
df_ag_production_agg = df_ag_production_agg[["Area", "Year", "frac_allocation"]].rename(columns = {"Area": "country", "Year": "year"})
df_data_to_allocate_wide = pd.merge(df_data_to_allocate_wide, df_ag_production_agg)
df_data_to_allocate_wide[field_qty] = np.array(df_data_to_allocate_wide[field_qty])*np.array(df_data_to_allocate_wide["frac_allocation"])
df_data_to_allocate_wide = df_data_to_allocate_wide[df_data_by_region_wide.columns]

# final dataset
df_data_out = pd.concat([df_data_by_region_wide, df_data_to_allocate_wide], axis = 0).sort_values(by = ["country", "year"]).reset_index(drop = True)
# functio to reformat the country name for integration
def format_country_name(country: str) -> str:
    country_out = country.split("(")[0].strip().lower().replace(" ", "_")
    return country_out
df_data_out["country"] = df_data_out["country"].apply(format_country_name)
fp_out = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/AFOLU/ifa_data_fertilizer_input_estimates_by_country_by_year.csv"
df_data_out.to_csv(fp_out, index = None, encoding = "UTF-8")

In [167]:
len(set(df_data_out["country"]))

26

In [161]:
("(")[0].strip().lower().replace(" ", "_")
    return country_out


In [178]:
df_frst_emission["Item"].unique()

array(['Forestland', 'Net Forest conversion',
       'Carbon stock change in forests'], dtype=object)

In [182]:
sorted(list(df_frst_emission["Item"].unique()))

['AFOLU',
 'Agricultural Soils',
 'Burning - Crop residues',
 'Crop Residues',
 'Drained organic soils',
 'Drained organic soils (CO2)',
 'Drained organic soils (N2O)',
 'Emissions on agricultural land',
 'Enteric Fermentation',
 'Farm-gate emissions',
 'Fires in humid tropical forests',
 'Fires in organic soils',
 'Forest fires',
 'Forestland',
 'IPCC Agriculture',
 'LULUCF',
 'Manure Management',
 'Manure applied to Soils',
 'Manure left on Pasture',
 'Net Forest conversion',
 'On-farm energy use',
 'Rice Cultivation',
 'Savanna fires',
 'Synthetic Fertilizers']

# SOME CODE FOR CALCULATING CROP FRACTIONS


In [77]:
constructs.time_periods#@sisepuede_file_struct.regions